In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D,Dropout,Bidirectional
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from gensim.models import Word2Vec

In [ ]:
from nltk.stem import  PorterStemmer

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train['text']=train['text'].apply(lambda x:clean_text(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
train['text']=train['text'].apply(lambda x:clean_text(x))

In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map 

In [ ]:
embeddings_dictionary = dict()
embedding_dim = 100
glove_file = open('../input/glovessss/glove.6B.100d.txt')
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = np.zeros((vocab_size,embedding_dim))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
words_to_index, index_to_words, word_to_vec_map=read_glove_vecs('../input/glovessss/glove.6B.100d.txt')

In [ ]:
embedding_dim = 100

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

In [ ]:
tokenizer=Tokenizer()
#total=X_train+X_test
tokenizer.fit_on_texts(train.text.values)

In [ ]:
maxlen=max([len(w.split()) for w in train.text.values ])

In [ ]:
maxlen

In [ ]:
vocab_size=len(tokenizer.word_index)+1

In [ ]:
def embed(corpus): 
    return tokenizer.texts_to_sequences(corpus)

In [ ]:
tweet=train.text.values
sentiment=train.target.values
padded_sentences=pad_sequences(embed(tweet),maxlen,padding='post')

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(padded_sentences,sentiment,test_size=0.15)

In [ ]:
from nltk.tokenize import word_tokenize
a="ohanaheshhitale"
long_train=max(train.text,key=lambda sent: len(word_tokenize(sent)))

In [ ]:
word=max(train.text,key=lambda x:len(word_tokenize(x)))
words=len(word_tokenize(word))

In [ ]:
tokenizer.word_index.items()

In [ ]:
emd_matrix=np.zeros((vocab_size,embedding_dim))

for word, i in tokenizer.word_index.items():
    embeding_vector=embeddings_dictionary.get(word)
    if embeding_vector is not None:
        emd_matrix[i]=embedding_vector
        
    
    

In [ ]:
emd_matrix.shape

In [ ]:
from keras.layers import BatchNormalization,GlobalMaxPool1D

In [ ]:
emd_dim=100
Model=Sequential()
Model.add(Embedding(input_dim=vocab_size,output_dim=emd_dim,input_length=maxlen,weights=[embedding_matrix]))
Model.add(Bidirectional(LSTM(units=maxlen,return_sequences = True,recurrent_dropout=0.2)))
Model.add(GlobalMaxPool1D())
Model.add(BatchNormalization())
Model.add(Dropout(0.5))
Model.add(Dense(maxlen,activation='relu'))
Model.add(Dropout(0.5))
Model.add(Dense(maxlen,activation='relu'))
Model.add(Dropout(0.5))
Model.add(Dense(1,activation='sigmoid'))
Model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
Model.summary()

In [ ]:
Model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32,verbose=1)

In [ ]:
Model.fit(padded_sentences,sentiment,epochs=15,batch_size=32)

In [ ]:
longest_train = max(tweet, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

In [ ]:
test

In [ ]:
a=test.text.values
padded_senetences_test=pad_sequences(embed(a),maxlen,padding='post')

In [ ]:
target=Model.predict_classes(padded_senetences_test)

In [ ]:
test['target'].value_counts()

In [ ]:
target

In [ ]:
test['target']=target
test=test[['id','target']]

In [ ]:
test['target'].value_counts()

In [ ]:
test.to_csv('Reaaal.csv',index=False)